In [14]:
import pandas as pd
data = pd.read_csv('customer_segmentation.csv')
data.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,payment_type,payment_installments,...,seller_id,shipping_limit_date,price,freight_value,seller_city,seller_state,product_category_name,product_name_lenght,product_description_lenght,product_category_name_english
0,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,boleto,1,...,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.7,22.76,belo horizonte,SP,perfumaria,29,178,perfumery
1,86674ccaee19790309333210917b2c7d,1b338293f35549b5e480b9a3d7bbf3cd,delivered,2018-08-09 11:37:35,2018-08-09 14:35:19,2018-08-10 14:34:00,2018-08-14 18:51:47,2018-08-22 00:00:00,credit_card,5,...,289cdb325fb7e7f891c38608bf9e0962,2018-08-13 14:31:29,116.9,18.92,belo horizonte,SP,perfumaria,29,178,perfumery
2,aee682982e18eb4714ce9f97b15af5e2,8858442ea4d5dc5bb9e118e8f728095d,delivered,2018-07-09 18:46:28,2018-07-11 03:45:45,2018-07-11 15:01:00,2018-07-12 18:14:35,2018-07-18 00:00:00,boleto,1,...,289cdb325fb7e7f891c38608bf9e0962,2018-07-13 03:45:45,118.7,9.34,belo horizonte,SP,perfumaria,29,178,perfumery
3,d543201a9b42a1402ff97e65b439a48b,971bf8f42a9f8cb3ead257854905b454,delivered,2018-08-21 10:00:25,2018-08-21 10:50:54,2018-08-22 15:21:00,2018-08-28 18:58:22,2018-09-10 00:00:00,credit_card,2,...,289cdb325fb7e7f891c38608bf9e0962,2018-08-23 10:50:54,116.9,22.75,belo horizonte,SP,perfumaria,29,178,perfumery
4,d543201a9b42a1402ff97e65b439a48b,971bf8f42a9f8cb3ead257854905b454,delivered,2018-08-21 10:00:25,2018-08-21 10:50:54,2018-08-22 15:21:00,2018-08-28 18:58:22,2018-09-10 00:00:00,credit_card,2,...,289cdb325fb7e7f891c38608bf9e0962,2018-08-23 10:50:54,116.9,22.75,belo horizonte,SP,perfumaria,29,178,perfumery


In [32]:
import mysql.connector as mysql
from mysql.connector import Error 

db_name='CustomersDatabaseFinal'
try:
    mydb = mysql.connect(host='localhost', user='Alessandro', password='Terracina1!', auth_plugin='mysql_native_password') # you can add the auth_plugin here too (ref line 26)
    if mydb.is_connected():
        mycursor = mydb.cursor()
        mycursor.execute('SHOW DATABASES')
        result = mycursor.fetchall()
        print(result)
        for x in result:
            if db_name == x[0]:
                mycursor.execute('DROP DATABASE ' + db_name) # delete old database
                mydb.commit() # make the changes official
                print("The database already exists! The old database has been deleted!)")
        
        mycursor.execute("CREATE DATABASE "+ db_name)
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)



[('customersdatabase',), ('customersdatabase2',), ('goshare',), ('information_schema',), ('mysql',), ('performance_schema',), ('sakila',), ('sys',), ('world',)]
Database is created


In [4]:
dataset = pd.read_csv('customer_segmentation.csv')
dataset.head()
dataset['payment_value'][0]

141.46

In [35]:
mycursor.execute("USE CustomersDatabaseFinal")


In [36]:
mycursor.execute(
      ''' 
        CREATE TABLE customer(
            customer_unique_id VARCHAR(40),
            customer_id VARCHAR(40),
            customer_city VARCHAR(40),
            customer_state VARCHAR(40),
            PRIMARY KEY (customer_unique_id)
            );
      '''
      )

In [37]:
mycursor.execute(
    '''
       CREATE TABLE seller(
            seller_id VARCHAR(40),
            seller_city VARCHAR(40),
            seller_state VARCHAR(40),
            PRIMARY KEY (seller_id)
            );


    '''





)

In [38]:
mycursor.execute(
    '''
       CREATE TABLE product(
            product_id VARCHAR(40),
            price int,
            freight_value int,
            product_category_name VARCHAR(60),
            product_description_lenght int,
            PRIMARY KEY (product_id)
            );


    '''





)

In [39]:


mycursor.execute(
    '''
        CREATE TABLE order_(
            order_id VARCHAR(40),
            order_status VARCHAR(40),
            order_purchase_time DATETIME,
            order_approved_at DATETIME,
            order_delivered_carrier_date DATETIME,
            order_delivered_customer_date DATETIME,
            order_estimated_delivery_date DATETIME,
            payment_type VARCHAR(40),
            payment_value int,
            payment_installments int,
            order_customer VARCHAR(40),
            order_product VARCHAR(40),
            order_seller VARCHAR(40),
            PRIMARY KEY (order_id),
            FOREIGN KEY (order_customer) REFERENCES customer(customer_unique_id),
            FOREIGN KEY (order_seller) REFERENCES seller(seller_id),
            FOREIGN KEY (order_product) REFERENCES product(product_id)
            ); 

    '''





)


In [40]:
mycursor.execute(
    '''
        CREATE TABLE offer(
            id VARCHAR(40),
            seller_id VARCHAR(40),
            product_id VARCHAR(40),
            PRIMARY KEY (id),
            FOREIGN KEY (seller_id) REFERENCES seller(seller_id),
            FOREIGN KEY (product_id) REFERENCES product(product_id)
        ); 
    '''





)

In [44]:
for i,row in data.iterrows():
    sql = "INSERT IGNORE INTO CustomersDatabaseFinal.customer VALUES (%s,%s,%s,%s)"
    mycursor.execute(sql, tuple([row['customer_unique_id'], row['customer_id'], row['customer_city'], row['customer_state']]))
    #print("Record inserted")
    mydb.commit()

In [45]:
for i,row in data.iterrows():
    sql = "INSERT IGNORE INTO CustomersDatabaseFinal.seller VALUES (%s,%s,%s)"
    mycursor.execute(sql, tuple([row['seller_id'], row['seller_city'], row['seller_state']]))
    #print("Record inserted")
    mydb.commit()



In [9]:
df2 = data.loc[:,['product_id','seller_id']]
df2.drop_duplicates(inplace = True)

In [ ]:
mycursor.execute("ALTER TABLE product DROP COLUMN product_seller")

In [46]:
for i,row in data.iterrows():
    sql = "INSERT IGNORE INTO CustomersDatabaseFinal.product VALUES (%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple([row['product_id'], row['price'], row['freight_value'], row['product_category_name'], row['product_description_lenght']]))
    #print("Record inserted")
    mydb.commit()
    

In [52]:

for i,row in data.iterrows():
    sql = "INSERT IGNORE INTO CustomersDatabaseFinal.order_ VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple([row['order_id'], row['order_status'], row['order_purchase_timestamp'], row['order_approved_at'], row['order_delivered_carrier_date'], row['order_delivered_customer_date'], row['order_estimated_delivery_date'], row['payment_type'], row['payment_value'], row['payment_installments'], row['customer_unique_id'], row['product_id'] , row['seller_id']]))
    #print("Record inserted")
    mydb.commit()

In [50]:
c = 0
for i,row in df2.iterrows():
    sql = "INSERT IGNORE INTO CustomersDatabaseFinal.offer VALUES (%s,%s,%s)"
    mycursor.execute(sql, tuple([c, row['seller_id'], row['product_id']]))
    c+=1
    #print("Record inserted")
    mydb.commit()

ProgrammingError: 1146 (42S02): Table 'customersdatabase.offers' doesn't exist